# Week 3: Advanced SQL for Redshift

## Learning Objectives
- Master Redshift-specific SQL features
- Optimize queries using distribution and sort keys
- Analyze and improve query performance
- Use EXPLAIN to understand query plans
- Implement advanced window functions at scale
- Leverage columnar storage benefits
- Design efficient joins for large tables
- Create and use materialized views

## Prerequisites
```bash
pip install pandas psycopg2-binary sqlalchemy ipython-sql
```

## 1. Setup and Connection

In [ ]:
import os
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text
import logging
import time
from typing import Dict, List

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Redshift configuration
REDSHIFT_CONFIG = {
    'host': os.getenv('REDSHIFT_HOST', 'your-cluster.region.redshift.amazonaws.com'),
    'port': int(os.getenv('REDSHIFT_PORT', 5439)),
    'database': os.getenv('REDSHIFT_DB', 'marketing'),
    'user': os.getenv('REDSHIFT_USER', 'analyst'),
    'password': os.getenv('REDSHIFT_PASSWORD', 'your-password')
}

# Create connection string
connection_string = (
    f"postgresql+psycopg2://{REDSHIFT_CONFIG['user']}:{REDSHIFT_CONFIG['password']}"
    f"@{REDSHIFT_CONFIG['host']}:{REDSHIFT_CONFIG['port']}/{REDSHIFT_CONFIG['database']}"
)

# Create engine
engine = create_engine(connection_string, pool_pre_ping=True)

def execute_query(query: str, fetch: bool = True):
    """Execute query and return results."""
    start_time = time.time()
    
    with engine.connect() as conn:
        result = conn.execute(text(query))
        
        if fetch:
            df = pd.DataFrame(result.fetchall(), columns=result.keys())
            elapsed = time.time() - start_time
            logger.info(f"Query returned {len(df):,} rows in {elapsed:.2f}s")
            return df
        else:
            conn.commit()
            elapsed = time.time() - start_time
            logger.info(f"Query executed in {elapsed:.2f}s")

print("✓ Connected to Redshift")

## 2. Redshift Table Design

### 2.1 Distribution Styles

In [ ]:
# Distribution style examples

# KEY Distribution - Distribute based on a column (for joins)
create_events_table = """
CREATE TABLE IF NOT EXISTS marketing_events (
    event_id BIGINT,
    user_id INTEGER,
    campaign_id INTEGER,
    channel VARCHAR(50),
    event_type VARCHAR(50),
    revenue DECIMAL(10,2),
    timestamp TIMESTAMP,
    date DATE
)
DISTKEY(user_id)  -- Distribute by user_id for user-based joins
SORTKEY(timestamp, user_id);  -- Sort by timestamp first, then user_id
"""

# EVEN Distribution - Distribute evenly (for tables not joined often)
create_campaigns_table = """
CREATE TABLE IF NOT EXISTS campaigns (
    campaign_id INTEGER,
    campaign_name VARCHAR(200),
    channel VARCHAR(50),
    budget DECIMAL(10,2),
    start_date DATE,
    end_date DATE
)
DISTSTYLE EVEN
SORTKEY(campaign_id);
"""

# ALL Distribution - Copy table to all nodes (for small dimension tables)
create_channels_table = """
CREATE TABLE IF NOT EXISTS channels (
    channel_id INTEGER,
    channel_name VARCHAR(50),
    category VARCHAR(50)
)
DISTSTYLE ALL;
"""

print("""
Distribution Style Guidelines:
------------------------------
KEY:  Use for large tables frequently joined on a specific column
EVEN: Use for tables that don't join often or join on different columns
ALL:  Use for small dimension tables (< 1-2 million rows) joined frequently
""")

### 2.2 Sort Keys

In [ ]:
# Compound Sort Key - Multiple columns in order
create_table_compound = """
CREATE TABLE user_sessions (
    session_id BIGINT,
    user_id INTEGER,
    session_date DATE,
    channel VARCHAR(50),
    duration_seconds INTEGER
)
DISTKEY(user_id)
COMPOUND SORTKEY(session_date, user_id);  -- Date first for date range queries
"""

# Interleaved Sort Key - Equal weight to all columns
create_table_interleaved = """
CREATE TABLE conversions (
    conversion_id BIGINT,
    user_id INTEGER,
    campaign_id INTEGER,
    conversion_date DATE,
    revenue DECIMAL(10,2)
)
DISTKEY(user_id)
INTERLEAVED SORTKEY(user_id, campaign_id, conversion_date);  -- Query on any combination
"""

print("""
Sort Key Guidelines:
--------------------
COMPOUND:     Use when you have a clear query pattern (e.g., always filter by date)
INTERLEAVED:  Use when queries filter on different column combinations
              Note: Interleaved is more expensive to maintain

Sort Key Column Order (Compound):
1. Columns used in WHERE clauses (especially date ranges)
2. Columns used in JOIN conditions
3. Columns used in GROUP BY
""")

## 3. Query Performance Analysis

### 3.1 Using EXPLAIN

In [ ]:
def analyze_query_plan(query: str) -> pd.DataFrame:
    """Get and display EXPLAIN plan."""
    explain_query = f"EXPLAIN {query}"
    return execute_query(explain_query)

# Example query to analyze
sample_query = """
SELECT 
    channel,
    DATE_TRUNC('day', timestamp) as date,
    COUNT(*) as events,
    SUM(revenue) as total_revenue
FROM marketing_events
WHERE timestamp >= '2024-01-01'
  AND timestamp < '2024-02-01'
GROUP BY channel, DATE_TRUNC('day', timestamp)
"""

# Get query plan
# plan = analyze_query_plan(sample_query)
# print(plan)

print("""
EXPLAIN Output - What to Look For:
----------------------------------
1. DS_DIST_ALL_NONE:    Good - No data redistribution needed
2. DS_DIST_BOTH:        Bad - Data redistributed on both sides of join
3. DS_BCAST_INNER:      OK - Small table broadcasted to all nodes
4. Seq Scan:            Bad - Full table scan (missing where clause or index)
5. Hash Join:           Good - Efficient join method
6. Nested Loop:         Bad - Inefficient for large datasets
""")

### 3.2 Query Performance Monitoring

In [ ]:
# Query to find slow queries
slow_queries = """
SELECT 
    query,
    TRIM(querytxt) as sql_text,
    starttime,
    endtime,
    DATEDIFF(seconds, starttime, endtime) as duration_seconds,
    aborted,
    userid,
    database
FROM stl_query
WHERE starttime >= DATEADD(hour, -24, GETDATE())
  AND DATEDIFF(seconds, starttime, endtime) > 60  -- Queries longer than 60s
  AND aborted = 0
ORDER BY duration_seconds DESC
LIMIT 20;
"""

# Query to check table statistics
table_stats = """
SELECT 
    "schema",
    "table",
    size,
    tbl_rows,
    unsorted,
    stats_off
FROM svv_table_info
WHERE "schema" NOT IN ('pg_catalog', 'information_schema')
ORDER BY size DESC
LIMIT 20;
"""

# Query to check distribution skew
distribution_skew = """
SELECT 
    slice,
    COUNT(*) as row_count
FROM marketing_events
GROUP BY slice
ORDER BY row_count DESC;
"""

print("""
Performance Monitoring Queries:
-------------------------------
1. STL_QUERY:        Historical query performance
2. SVV_TABLE_INFO:   Table size, rows, and statistics
3. Distribution:     Check for data skew across slices
4. STL_ALERT_EVENT_LOG: Alerts for performance issues
""")

## 4. Advanced Window Functions at Scale

In [ ]:
# Example 1: Running Totals
running_total_query = """
SELECT 
    user_id,
    timestamp,
    revenue,
    SUM(revenue) OVER (
        PARTITION BY user_id 
        ORDER BY timestamp
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) as cumulative_revenue
FROM marketing_events
WHERE event_type = 'conversion'
ORDER BY user_id, timestamp;
"""

# Example 2: Moving Averages
moving_average_query = """
SELECT 
    DATE_TRUNC('day', timestamp) as date,
    channel,
    SUM(revenue) as daily_revenue,
    AVG(SUM(revenue)) OVER (
        PARTITION BY channel 
        ORDER BY DATE_TRUNC('day', timestamp)
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) as revenue_7d_ma
FROM marketing_events
GROUP BY DATE_TRUNC('day', timestamp), channel
ORDER BY channel, date;
"""

# Example 3: Ranking and Row Numbers
ranking_query = """
SELECT 
    user_id,
    campaign_id,
    timestamp,
    revenue,
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY timestamp) as event_sequence,
    RANK() OVER (PARTITION BY campaign_id ORDER BY revenue DESC) as revenue_rank,
    PERCENT_RANK() OVER (PARTITION BY campaign_id ORDER BY revenue) as revenue_percentile
FROM marketing_events
WHERE event_type = 'conversion';
"""

# Example 4: Lead and Lag (Next/Previous Values)
lead_lag_query = """
SELECT 
    user_id,
    timestamp as current_event_time,
    channel as current_channel,
    LAG(channel) OVER (PARTITION BY user_id ORDER BY timestamp) as previous_channel,
    LEAD(channel) OVER (PARTITION BY user_id ORDER BY timestamp) as next_channel,
    DATEDIFF(
        second, 
        LAG(timestamp) OVER (PARTITION BY user_id ORDER BY timestamp),
        timestamp
    ) as seconds_since_last_event
FROM marketing_events
ORDER BY user_id, timestamp;
"""

# Example 5: First/Last Value
first_last_query = """
SELECT DISTINCT
    user_id,
    FIRST_VALUE(channel) OVER (
        PARTITION BY user_id 
        ORDER BY timestamp
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) as first_touch_channel,
    LAST_VALUE(channel) OVER (
        PARTITION BY user_id 
        ORDER BY timestamp
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) as last_touch_channel,
    COUNT(*) OVER (PARTITION BY user_id) as total_touchpoints
FROM marketing_events;
"""

print("""
Window Function Best Practices:
-------------------------------
1. Use PARTITION BY to limit the window scope
2. Specify explicit frame clauses for clarity
3. Consider sort key alignment with ORDER BY
4. Use ROWS vs RANGE appropriately:
   - ROWS: Physical row positions
   - RANGE: Logical value ranges
5. Avoid multiple window functions with different partitions in one query
""")

## 5. Optimized Joins for Large Tables

In [ ]:
# Example 1: Optimized JOIN with matching distribution keys
optimized_join = """
-- Both tables distributed by user_id - No redistribution needed
SELECT 
    e.user_id,
    e.campaign_id,
    e.revenue,
    u.segment,
    u.signup_date
FROM marketing_events e
INNER JOIN users u ON e.user_id = u.user_id  -- Same distkey
WHERE e.date >= '2024-01-01';
"""

# Example 2: Join with small dimension table (DISTSTYLE ALL)
dimension_join = """
-- Campaigns table is DISTSTYLE ALL, copied to all nodes
SELECT 
    e.campaign_id,
    c.campaign_name,
    c.channel,
    COUNT(*) as events,
    SUM(e.revenue) as total_revenue
FROM marketing_events e
INNER JOIN campaigns c ON e.campaign_id = c.campaign_id
GROUP BY e.campaign_id, c.campaign_name, c.channel;
"""

# Example 3: Using subqueries to reduce join size
filtered_join = """
-- Filter before joining to reduce data volume
WITH recent_events AS (
    SELECT user_id, campaign_id, SUM(revenue) as revenue
    FROM marketing_events
    WHERE date >= '2024-01-01'
    GROUP BY user_id, campaign_id
),
high_value_users AS (
    SELECT user_id, segment
    FROM users
    WHERE segment IN ('high', 'vip')
)
SELECT 
    e.user_id,
    e.campaign_id,
    e.revenue,
    u.segment
FROM recent_events e
INNER JOIN high_value_users u ON e.user_id = u.user_id;
"""

# Example 4: Self-join for sequential analysis
self_join_query = """
-- Find users who converted after seeing an ad
SELECT 
    impressions.user_id,
    impressions.timestamp as impression_time,
    conversions.timestamp as conversion_time,
    DATEDIFF(second, impressions.timestamp, conversions.timestamp) as time_to_conversion
FROM marketing_events impressions
INNER JOIN marketing_events conversions
    ON impressions.user_id = conversions.user_id
    AND conversions.timestamp > impressions.timestamp
    AND conversions.timestamp <= DATEADD(day, 7, impressions.timestamp)
WHERE impressions.event_type = 'impression'
  AND conversions.event_type = 'conversion';
"""

print("""
Join Optimization Guidelines:
----------------------------
1. Match distribution keys for large table joins
2. Use DISTSTYLE ALL for small dimension tables
3. Filter data before joining (in CTEs or subqueries)
4. Put largest table first in FROM clause
5. Use INNER JOIN when possible (more efficient than OUTER)
6. Avoid joining on expressions - join on columns directly
7. Consider materializing complex join results
""")

## 6. Columnar Storage Benefits

In [ ]:
# Example 1: Selective column reading (fast with columnar storage)
columnar_benefit_1 = """
-- Only reads 3 columns from disk (not full rows)
SELECT 
    user_id,
    campaign_id,
    revenue
FROM marketing_events
WHERE date >= '2024-01-01';
"""

# Example 2: Compression benefits
check_compression = """
-- Check column compression encodings
SELECT 
    "column",
    type,
    encoding,
    distkey,
    sortkey
FROM pg_table_def
WHERE tablename = 'marketing_events';
"""

# Example 3: Optimize with column encodings
create_with_encoding = """
CREATE TABLE marketing_events_optimized (
    event_id BIGINT ENCODE DELTA,           -- Delta encoding for sequential IDs
    user_id INTEGER ENCODE DELTA32K,        -- Delta for IDs
    campaign_id INTEGER ENCODE RUNLENGTH,   -- Runlength if sorted by campaign
    channel VARCHAR(50) ENCODE LZO,         -- LZO for text
    event_type VARCHAR(50) ENCODE BYTEDICT, -- Dictionary for low cardinality
    revenue DECIMAL(10,2) ENCODE DELTA32K,
    timestamp TIMESTAMP ENCODE DELTA32K,
    date DATE ENCODE DELTA32K
)
DISTKEY(user_id)
SORTKEY(date, user_id);
"""

# Example 4: Analyze compression
analyze_compression_query = """
ANALYZE COMPRESSION marketing_events;
"""

print("""
Columnar Storage Benefits:
-------------------------
1. Only read columns needed (not full rows)
2. Better compression (similar values together)
3. Faster aggregations on specific columns
4. Efficient column statistics (min/max zone maps)

Compression Encoding Types:
--------------------------
RAW:       No compression
DELTA:     Good for sorted numeric columns
DELTA32K:  Delta with 32K block size
LZO:       General purpose compression
BYTEDICT:  Dictionary for low cardinality (<50K unique)
RUNLENGTH: Good for sorted columns with runs of same value
ZSTD:      High compression ratio (newer)
""")

## 7. Materialized Views

In [ ]:
# Create a materialized view for daily campaign metrics
create_mv_daily_metrics = """
CREATE MATERIALIZED VIEW mv_daily_campaign_metrics
DISTKEY(campaign_id)
SORTKEY(date)
AS
SELECT 
    campaign_id,
    DATE_TRUNC('day', timestamp) as date,
    channel,
    COUNT(*) as total_events,
    COUNT(DISTINCT user_id) as unique_users,
    SUM(CASE WHEN event_type = 'impression' THEN 1 ELSE 0 END) as impressions,
    SUM(CASE WHEN event_type = 'click' THEN 1 ELSE 0 END) as clicks,
    SUM(CASE WHEN event_type = 'conversion' THEN 1 ELSE 0 END) as conversions,
    SUM(revenue) as total_revenue,
    AVG(revenue) as avg_revenue
FROM marketing_events
GROUP BY campaign_id, DATE_TRUNC('day', timestamp), channel;
"""

# Refresh materialized view
refresh_mv = """
REFRESH MATERIALIZED VIEW mv_daily_campaign_metrics;
"""

# Query using materialized view (much faster)
query_mv = """
SELECT 
    campaign_id,
    channel,
    SUM(impressions) as total_impressions,
    SUM(clicks) as total_clicks,
    SUM(conversions) as total_conversions,
    SUM(total_revenue) as revenue,
    SUM(clicks) * 1.0 / NULLIF(SUM(impressions), 0) as ctr,
    SUM(conversions) * 1.0 / NULLIF(SUM(clicks), 0) as cvr
FROM mv_daily_campaign_metrics
WHERE date >= '2024-01-01'
  AND date < '2024-02-01'
GROUP BY campaign_id, channel;
"""

# Auto-refresh materialized view (if supported)
create_auto_refresh_mv = """
CREATE MATERIALIZED VIEW mv_hourly_metrics
AUTO REFRESH YES
AS
SELECT 
    DATE_TRUNC('hour', timestamp) as hour,
    channel,
    COUNT(*) as events,
    SUM(revenue) as revenue
FROM marketing_events
WHERE timestamp >= DATEADD(day, -7, GETDATE())
GROUP BY DATE_TRUNC('hour', timestamp), channel;
"""

print("""
Materialized View Best Practices:
---------------------------------
1. Use for frequently-run expensive aggregations
2. Add distribution and sort keys to MVs
3. Refresh on a schedule or use auto-refresh
4. Monitor MV staleness
5. Consider incremental refresh if available
6. Don't over-use (storage cost vs query benefit)
""")

## 8. Complex Analytics Queries

In [ ]:
# Example 1: Cohort Analysis
cohort_analysis = """
WITH user_cohorts AS (
    SELECT 
        user_id,
        DATE_TRUNC('month', MIN(timestamp)) as cohort_month
    FROM marketing_events
    WHERE event_type = 'conversion'
    GROUP BY user_id
),
user_activity AS (
    SELECT 
        c.cohort_month,
        DATE_TRUNC('month', e.timestamp) as activity_month,
        COUNT(DISTINCT e.user_id) as active_users,
        SUM(e.revenue) as revenue
    FROM user_cohorts c
    INNER JOIN marketing_events e ON c.user_id = e.user_id
    WHERE e.event_type = 'conversion'
    GROUP BY c.cohort_month, DATE_TRUNC('month', e.timestamp)
)
SELECT 
    cohort_month,
    activity_month,
    DATEDIFF(month, cohort_month, activity_month) as months_since_first_conversion,
    active_users,
    revenue,
    revenue / active_users as revenue_per_user
FROM user_activity
ORDER BY cohort_month, activity_month;
"""

# Example 2: Funnel Analysis
funnel_analysis = """
WITH user_events AS (
    SELECT 
        user_id,
        MAX(CASE WHEN event_type = 'impression' THEN 1 ELSE 0 END) as has_impression,
        MAX(CASE WHEN event_type = 'click' THEN 1 ELSE 0 END) as has_click,
        MAX(CASE WHEN event_type = 'cart_add' THEN 1 ELSE 0 END) as has_cart_add,
        MAX(CASE WHEN event_type = 'conversion' THEN 1 ELSE 0 END) as has_conversion
    FROM marketing_events
    WHERE date >= '2024-01-01'
      AND date < '2024-02-01'
    GROUP BY user_id
)
SELECT 
    SUM(has_impression) as impressions,
    SUM(has_click) as clicks,
    SUM(has_cart_add) as cart_adds,
    SUM(has_conversion) as conversions,
    SUM(has_click) * 100.0 / NULLIF(SUM(has_impression), 0) as impression_to_click_rate,
    SUM(has_cart_add) * 100.0 / NULLIF(SUM(has_click), 0) as click_to_cart_rate,
    SUM(has_conversion) * 100.0 / NULLIF(SUM(has_cart_add), 0) as cart_to_conversion_rate
FROM user_events;
"""

# Example 3: RFM Segmentation
rfm_segmentation = """
WITH user_metrics AS (
    SELECT 
        user_id,
        DATEDIFF(day, MAX(timestamp), GETDATE()) as recency_days,
        COUNT(*) as frequency,
        SUM(revenue) as monetary
    FROM marketing_events
    WHERE event_type = 'conversion'
    GROUP BY user_id
),
rfm_scores AS (
    SELECT 
        user_id,
        recency_days,
        frequency,
        monetary,
        NTILE(5) OVER (ORDER BY recency_days) as r_score,
        NTILE(5) OVER (ORDER BY frequency DESC) as f_score,
        NTILE(5) OVER (ORDER BY monetary DESC) as m_score
    FROM user_metrics
)
SELECT 
    CASE 
        WHEN r_score >= 4 AND f_score >= 4 AND m_score >= 4 THEN 'Champions'
        WHEN r_score >= 3 AND f_score >= 3 AND m_score >= 3 THEN 'Loyal Customers'
        WHEN r_score >= 4 AND f_score <= 2 THEN 'New Customers'
        WHEN r_score <= 2 AND f_score >= 3 THEN 'At Risk'
        WHEN r_score <= 2 AND f_score <= 2 THEN 'Lost'
        ELSE 'Other'
    END as segment,
    COUNT(*) as user_count,
    AVG(recency_days) as avg_recency,
    AVG(frequency) as avg_frequency,
    AVG(monetary) as avg_monetary
FROM rfm_scores
GROUP BY segment
ORDER BY user_count DESC;
"""

# Example 4: Time-Series Anomaly Detection
anomaly_detection = """
WITH daily_metrics AS (
    SELECT 
        DATE_TRUNC('day', timestamp) as date,
        channel,
        SUM(revenue) as daily_revenue
    FROM marketing_events
    WHERE timestamp >= DATEADD(day, -90, GETDATE())
    GROUP BY DATE_TRUNC('day', timestamp), channel
),
stats AS (
    SELECT 
        date,
        channel,
        daily_revenue,
        AVG(daily_revenue) OVER (
            PARTITION BY channel 
            ORDER BY date 
            ROWS BETWEEN 29 PRECEDING AND 1 PRECEDING
        ) as avg_30d,
        STDDEV(daily_revenue) OVER (
            PARTITION BY channel 
            ORDER BY date 
            ROWS BETWEEN 29 PRECEDING AND 1 PRECEDING
        ) as stddev_30d
    FROM daily_metrics
)
SELECT 
    date,
    channel,
    daily_revenue,
    avg_30d,
    CASE 
        WHEN daily_revenue > avg_30d + 2 * stddev_30d THEN 'High Anomaly'
        WHEN daily_revenue < avg_30d - 2 * stddev_30d THEN 'Low Anomaly'
        ELSE 'Normal'
    END as anomaly_flag,
    (daily_revenue - avg_30d) / NULLIF(stddev_30d, 0) as z_score
FROM stats
WHERE date >= DATEADD(day, -30, GETDATE())
ORDER BY date DESC, channel;
"""

print("✓ Advanced analytics queries defined")

## 9. Real-World Project: Build Marketing Data Warehouse

### Complete data warehouse schema design

In [ ]:
# Step 1: Create fact table (events)
create_fact_events = """
CREATE TABLE IF NOT EXISTS fact_marketing_events (
    event_id BIGINT NOT NULL,
    user_id INTEGER NOT NULL,
    campaign_id INTEGER,
    channel_id INTEGER,
    event_type_id INTEGER,
    timestamp TIMESTAMP NOT NULL,
    date DATE NOT NULL,
    revenue DECIMAL(10,2),
    cost DECIMAL(10,2),
    session_id BIGINT,
    device_type VARCHAR(50),
    location_id INTEGER
)
DISTKEY(user_id)
SORTKEY(date, timestamp)
;
"""

# Step 2: Create dimension tables
create_dim_users = """
CREATE TABLE IF NOT EXISTS dim_users (
    user_id INTEGER NOT NULL,
    email_hash VARCHAR(64),
    signup_date DATE,
    segment VARCHAR(50),
    country VARCHAR(2),
    acquisition_channel VARCHAR(50),
    lifetime_value DECIMAL(10,2),
    is_active BOOLEAN
)
DISTKEY(user_id)
SORTKEY(user_id)
;
"""

create_dim_campaigns = """
CREATE TABLE IF NOT EXISTS dim_campaigns (
    campaign_id INTEGER NOT NULL,
    campaign_name VARCHAR(200),
    campaign_type VARCHAR(50),
    channel_id INTEGER,
    start_date DATE,
    end_date DATE,
    budget DECIMAL(10,2),
    objective VARCHAR(50)
)
DISTSTYLE ALL
SORTKEY(campaign_id)
;
"""

create_dim_channels = """
CREATE TABLE IF NOT EXISTS dim_channels (
    channel_id INTEGER NOT NULL,
    channel_name VARCHAR(50),
    channel_category VARCHAR(50),
    is_paid BOOLEAN
)
DISTSTYLE ALL
;
"""

# Step 3: Create aggregated fact table
create_fact_daily_summary = """
CREATE TABLE IF NOT EXISTS fact_daily_campaign_summary (
    date DATE NOT NULL,
    campaign_id INTEGER NOT NULL,
    channel_id INTEGER,
    impressions INTEGER,
    clicks INTEGER,
    conversions INTEGER,
    unique_users INTEGER,
    revenue DECIMAL(12,2),
    cost DECIMAL(12,2),
    PRIMARY KEY (date, campaign_id)
)
DISTKEY(campaign_id)
SORTKEY(date)
;
"""

# Step 4: ETL query to populate daily summary
populate_daily_summary = """
INSERT INTO fact_daily_campaign_summary
SELECT 
    date,
    campaign_id,
    channel_id,
    SUM(CASE WHEN event_type_id = 1 THEN 1 ELSE 0 END) as impressions,
    SUM(CASE WHEN event_type_id = 2 THEN 1 ELSE 0 END) as clicks,
    SUM(CASE WHEN event_type_id = 3 THEN 1 ELSE 0 END) as conversions,
    COUNT(DISTINCT user_id) as unique_users,
    SUM(revenue) as revenue,
    SUM(cost) as cost
FROM fact_marketing_events
WHERE date = CURRENT_DATE - 1  -- Yesterday's data
GROUP BY date, campaign_id, channel_id;
"""

# Step 5: Create reporting views
create_campaign_performance_view = """
CREATE OR REPLACE VIEW vw_campaign_performance AS
SELECT 
    f.date,
    c.campaign_name,
    ch.channel_name,
    f.impressions,
    f.clicks,
    f.conversions,
    f.unique_users,
    f.revenue,
    f.cost,
    f.revenue - f.cost as profit,
    f.clicks * 100.0 / NULLIF(f.impressions, 0) as ctr,
    f.conversions * 100.0 / NULLIF(f.clicks, 0) as cvr,
    f.cost / NULLIF(f.clicks, 0) as cpc,
    f.cost / NULLIF(f.conversions, 0) as cpa,
    f.revenue / NULLIF(f.cost, 0) as roas
FROM fact_daily_campaign_summary f
LEFT JOIN dim_campaigns c ON f.campaign_id = c.campaign_id
LEFT JOIN dim_channels ch ON f.channel_id = ch.channel_id;
"""

print("""
Data Warehouse Design Best Practices:
------------------------------------
1. Use star schema (fact + dimensions)
2. Distribute large fact tables by commonly joined keys
3. Use DISTSTYLE ALL for small dimension tables
4. Create pre-aggregated fact tables for common queries
5. Use appropriate data types to minimize storage
6. Implement incremental loading strategies
7. Create views for common business logic
8. Document table structures and dependencies
""")

## 10. Maintenance and Optimization

In [ ]:
# Vacuum to reclaim space and resort
vacuum_table = """
VACUUM SORT ONLY marketing_events;  -- Resort rows
VACUUM DELETE ONLY marketing_events;  -- Reclaim space
VACUUM FULL marketing_events;  -- Both resort and reclaim
"""

# Analyze to update statistics
analyze_table = """
ANALYZE marketing_events;
"""

# Check for tables needing vacuum
check_vacuum_needed = """
SELECT 
    "schema",
    "table",
    unsorted,
    stats_off,
    tbl_rows
FROM svv_table_info
WHERE unsorted > 5  -- More than 5% unsorted
   OR stats_off > 5  -- Statistics more than 5% off
ORDER BY unsorted DESC;
"""

# Monitor running queries
running_queries = """
SELECT 
    pid,
    user_name,
    starttime,
    DATEDIFF(seconds, starttime, GETDATE()) as runtime_seconds,
    TRIM(query_text) as query
FROM stv_recents
WHERE status = 'Running'
ORDER BY runtime_seconds DESC;
"""

# Cancel a query
cancel_query = """
CANCEL <pid>;
"""

# Check WLM queue wait times
wlm_queue_state = """
SELECT 
    service_class,
    num_queued_queries,
    num_executing_queries,
    num_executed_queries
FROM stv_wlm_service_class_state
WHERE service_class >= 6;
"""

print("""
Maintenance Best Practices:
---------------------------
1. Run VACUUM regularly (weekly for active tables)
2. Run ANALYZE after significant data changes
3. Monitor table statistics (svv_table_info)
4. Check for data skew across slices
5. Monitor query performance (stl_query)
6. Set up alerts for slow queries
7. Review and optimize WLM configuration
8. Archive old data regularly
""")

## 11. Best Practices Summary

### Table Design
1. Choose appropriate distribution style (KEY, EVEN, ALL)
2. Select sort keys based on query patterns
3. Use compound sort keys for ordered queries
4. Use interleaved sort keys for varied queries
5. Optimize column encodings for compression

### Query Optimization
1. Use EXPLAIN to analyze query plans
2. Select only required columns
3. Filter early and filter often
4. Match distribution keys in joins
5. Use materialized views for complex aggregations

### Performance
1. Monitor query performance regularly
2. Check for data skew
3. Vacuum and analyze tables
4. Use appropriate WLM queues
5. Archive old data

### Cost Optimization
1. Use appropriate node types
2. Implement data lifecycle policies
3. Use Redshift Spectrum for cold data
4. Monitor and optimize storage
5. Pause clusters when not in use

## 12. Exercises

### Exercise 1: Table Design
Design a Redshift table schema for:
1. 100M row events table with optimal distribution and sort keys
2. Associated dimension tables
3. Appropriate column encodings
4. Justify your design choices

### Exercise 2: Query Optimization
1. Write a complex multi-table join query
2. Use EXPLAIN to analyze the query plan
3. Identify and fix performance issues
4. Benchmark before and after optimization

### Exercise 3: Window Functions
Implement the following using window functions:
1. Running totals by user
2. 7-day moving average by channel
3. First and last touch attribution
4. User journey sequence analysis

### Exercise 4: Data Warehouse
Build a complete data warehouse:
1. Design star schema
2. Create fact and dimension tables
3. Implement ETL process
4. Create reporting views
5. Set up maintenance procedures

## Resources

### Documentation
- [Redshift Best Practices](https://docs.aws.amazon.com/redshift/latest/dg/best-practices.html)
- [Redshift SQL Reference](https://docs.aws.amazon.com/redshift/latest/dg/c_SQL_reference.html)
- [Distribution Styles](https://docs.aws.amazon.com/redshift/latest/dg/c_choosing_dist_sort.html)
- [Window Functions](https://docs.aws.amazon.com/redshift/latest/dg/c_Window_functions.html)

### Tools
- AWS Redshift Console: Query editor and monitoring
- DBeaver: SQL IDE with Redshift support
- SQL Workbench/J: Redshift management tool
- CloudWatch: Performance monitoring